In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, row_number, col, desc, filter

In [0]:
df_employeepay = spark.read.parquet("/mnt/bronze/human_resources/employee_pay_history/20220104")
df_employeedepart = spark.read.parquet("/mnt/bronze/human_resources/employee_department_history/20211208")
df_department = spark.read.parquet("/mnt/bronze/human_resources/department/20220104")

In [0]:
windowSpec  = Window.partitionBy(col("BusinessEntityID")).orderBy(col("RateChangeDate").desc())
windowDepart = Window.partitionBy(col("BusinessEntityID")).orderBy(col("StartDate").desc())

In [0]:
df_pay = df_employeepay.withColumn("Rank_Salary",row_number().over(windowSpec))
#display(df_pay)

In [0]:
df_depart = df_employeedepart.withColumn("Rank_Depart",row_number().over(windowDepart))
#display(df_depart)

In [0]:
df_pay_rank = df_pay.filter(df_pay.Rank_Salary == "1")
#display(df_pay_rank)

In [0]:
df_depart_rank = df_depart.filter(df_depart.Rank_Depart == "1")
#display(df_depart_rank)

In [0]:
df_pay_rank.registerTempTable("pay")
df_depart_rank.registerTempTable("depart")
df_department.registerTempTable("department")

In [0]:
#display(df_department)

In [0]:
df_join = spark.sql("""
select 
pay.BusinessEntityID,
depart.DepartmentID,
department.Name NameDepartment,
department.GroupName,
depart.StartDate,
pay.Rate
from pay
left join depart
  on pay.BusinessEntityID = depart.BusinessEntityID
left join department
  on department.DepartmentID = depart.DepartmentID""")
display(df_join.limit(5))

BusinessEntityID,DepartmentID,NameDepartment,GroupName,StartDate,Rate
1,16,Executive,Executive General and Administration,2009-01-14,125.5000
2,1,Engineering,Research and Development,2008-01-31,63.4615
3,1,Engineering,Research and Development,2007-11-11,43.2692
4,2,Tool Design,Research and Development,2010-05-31,29.8462
5,1,Engineering,Research and Development,2008-01-06,32.6923
6,1,Engineering,Research and Development,2008-01-24,32.6923
7,6,Research and Development,Research and Development,2009-02-08,50.4808
8,6,Research and Development,Research and Development,2008-12-29,40.8654
9,6,Research and Development,Research and Development,2009-01-16,40.8654
10,6,Research and Development,Research and Development,2009-05-03,42.4808


In [0]:
df_join.write.format("Delta").save("/mnt/gold/human_resources/SalarioDepart")